In [3]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("notebooks/distilled_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets
from model_config import LSTM_Models
from compact.knowledge_distillation import KnowledgeDistillation

In [4]:
conf = Config()
load_models = LSTM_Models()
teacher_model = load_models.lstm_4
teacher_model.load()
student_model = load_models.light_lstm_1
kd = KnowledgeDistillation(
    teacher=teacher_model,
    student=student_model,
    device=conf.device,
    distillation="relation",
    rnn=True
)

Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/large_model/lstm_4.pth!


In [5]:
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "lstm")

# create train, val and test dataloaders
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, conf.batch_size, shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, conf.batch_size, shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, conf.batch_size)

In [6]:
data, labels = next(iter(train_loader))
print(data.shape, labels.shape)

torch.Size([512, 513, 1]) torch.Size([512])


In [7]:
acc, train_loss, val_loss = kd.train_kd(train_loader, val_loader, conf.epochs)

Epoch: 1/10, Accuracy: 71.11%, Train loss: 0.0094, Val loss: 0.8649
Epoch: 2/10, Accuracy: 76.77%, Train loss: 0.0051, Val loss: 0.5267
Epoch: 3/10, Accuracy: 79.66%, Train loss: 0.0036, Val loss: 0.3567
Epoch: 4/10, Accuracy: 83.89%, Train loss: 0.0029, Val loss: 0.2797
Epoch: 5/10, Accuracy: 86.84%, Train loss: 0.0025, Val loss: 0.2415
Epoch: 6/10, Accuracy: 88.03%, Train loss: 0.0023, Val loss: 0.2196
Epoch: 7/10, Accuracy: 88.64%, Train loss: 0.0022, Val loss: 0.2026
Epoch: 8/10, Accuracy: 89.25%, Train loss: 0.0021, Val loss: 0.1910
Epoch: 9/10, Accuracy: 89.60%, Train loss: 0.0020, Val loss: 0.1800
Epoch: 10/10, Accuracy: 89.65%, Train loss: 0.0019, Val loss: 0.1737


In [8]:
# test teacher accuracy
loss, accuracy = teacher_model.evaluate(test_loader)
print(f"Teacher accuracy: {100*accuracy:.2f}%")

Teacher accuracy: 96.64%


In [9]:
# test student accuracy
loss, accuracy = student_model.evaluate(test_loader)
print(f"Student accuracy: {100*accuracy:.2f}%")

Student accuracy: 89.67%


In [10]:
student_model.save()

Checkpoint saved at /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/compressed_model/light_lstm_1.pth
